In [3]:
import commonsense.conceptnet as kb
from commonsense.logical_classes import to_data_frame, parse_file_to_fact_list
from monitor.reasonableness_monitor import SnapshotMonitor


cn = kb.ConceptNet()

cn_monitor = SnapshotMonitor()  # labels=labels, data=data, rules=rules)
print("Made a ConceptNet monitor, now trying to explain")

facts = parse_file_to_fact_list("datasets/output_feb25.txt")
cn_monitor.explain_all_events(facts)

Made a ConceptNet monitor, now trying to explain
Explaining lookingAtPEvent100
  robot is observing a looking event
Explaining lookingAtPEvent1001
  robot is observing a looking event
Explaining lookingAtPEvent1003
  robot is observing a looking event
Explaining lookingAtPEvent1005
  robot is observing a looking event
Explaining lookingAtPEvent1007
  robot is observing a looking event
Explaining lookingAtPEvent1009
  robot is observing a looking event
Explaining lookingAtPEvent1011
  robot is observing a looking event
Explaining lookingAtPEvent1013
  robot is observing a looking event
Explaining lookingAtPEvent1015
  robot is observing a looking event
Explaining lookingAtPEvent1017
  robot is observing a looking event
Explaining lookingAtPEvent1019
  robot is observing a looking event
Explaining lookingAtPEvent102
  robot is observing a looking event
Explaining lookingAtPEvent1021
  robot is observing a looking event
Explaining lookingAtPEvent1023
  robot is observing a looking event
E

Explaining lookingAtPEvent378
  robot is observing a looking event
Explaining lookingAtPEvent38
  robot is observing a looking event
Explaining lookingAtPEvent381
  robot is observing a looking event
Explaining lookingAtPEvent383
  robot is observing a looking event
Explaining lookingAtPEvent385
  robot is observing a looking event
Explaining lookingAtPEvent389
  robot is observing a looking event
Explaining lookingAtPEvent391
  robot is observing a looking event
Explaining lookingAtPEvent393
  robot is observing a looking event
Explaining lookingAtPEvent395
  robot is observing a looking event
Explaining lookingAtPEvent397
  robot is observing a looking event
Explaining lookingAtPEvent40
  robot is observing a looking event
Explaining lookingAtPEvent403
  robot is observing a looking event
Explaining lookingAtPEvent409
  robot is observing a looking event
Explaining lookingAtPEvent413
  robot is observing a looking event
Explaining lookingAtPEvent415
  robot is observing a looking eve

Explaining lookingAtPEvent834
  robot is observing a looking event
Explaining lookingAtPEvent836
  robot is observing a looking event
Explaining lookingAtPEvent838
  robot is observing a looking event
Explaining lookingAtPEvent84
  robot is observing a looking event
Explaining lookingAtPEvent840
  robot is observing a looking event
Explaining lookingAtPEvent842
  robot is observing a looking event
Explaining lookingAtPEvent844
  robot is observing a looking event
Explaining lookingAtPEvent846
  robot is observing a looking event
Explaining lookingAtPEvent848
  robot is observing a looking event
Explaining lookingAtPEvent850
  robot is observing a looking event
Explaining lookingAtPEvent852
  robot is observing a looking event
Explaining lookingAtPEvent854
  robot is observing a looking event
Explaining lookingAtPEvent856
  robot is observing a looking event
Explaining lookingAtPEvent858
  robot is observing a looking event
Explaining lookingAtPEvent86
  robot is observing a looking eve

Explaining lookingInDEvent1060
  robot is observing a looking event
Explaining lookingInDEvent1062
  robot is observing a looking event
Explaining lookingInDEvent1064
  robot is observing a looking event
Explaining lookingInDEvent1066
  robot is observing a looking event
Explaining lookingInDEvent1068
  robot is observing a looking event
Explaining lookingInDEvent107
  robot is observing a looking event
Explaining lookingInDEvent1070
  robot is observing a looking event
Explaining lookingInDEvent1072
  robot is observing a looking event
Explaining lookingInDEvent1074
  robot is observing a looking event
Explaining lookingInDEvent1076
  robot is observing a looking event
Explaining lookingInDEvent1078
  robot is observing a looking event
Explaining lookingInDEvent1080
  robot is observing a looking event
Explaining lookingInDEvent1082
  robot is observing a looking event
Explaining lookingInDEvent1084
  robot is observing a looking event
Explaining lookingInDEvent1086
  robot is observi

  robot is observing a looking event
Explaining lookingInDEvent43
  robot is observing a looking event
Explaining lookingInDEvent431
  robot is observing a looking event
Explaining lookingInDEvent432
  robot is observing a looking event
Explaining lookingInDEvent437
  robot is observing a looking event
Explaining lookingInDEvent439
  robot is observing a looking event
Explaining lookingInDEvent441
  robot is observing a looking event
Explaining lookingInDEvent443
  robot is observing a looking event
Explaining lookingInDEvent445
  robot is observing a looking event
Explaining lookingInDEvent447
  robot is observing a looking event
Explaining lookingInDEvent45
  robot is observing a looking event
Explaining lookingInDEvent451
  robot is observing a looking event
Explaining lookingInDEvent453
  robot is observing a looking event
Explaining lookingInDEvent455
  robot is observing a looking event
Explaining lookingInDEvent457
  robot is observing a looking event
Explaining lookingInDEvent4

Explaining lookingInDEvent671
  robot is observing a looking event
Explaining lookingInDEvent673
  robot is observing a looking event
Explaining lookingInDEvent675
  robot is observing a looking event
Explaining lookingInDEvent677
  robot is observing a looking event
Explaining lookingInDEvent679
  robot is observing a looking event
Explaining lookingInDEvent681
  robot is observing a looking event
Explaining lookingInDEvent683
  robot is observing a looking event
Explaining lookingInDEvent685
  robot is observing a looking event
Explaining lookingInDEvent687
  robot is observing a looking event
Explaining lookingInDEvent689
  robot is observing a looking event
Explaining lookingInDEvent69
  robot is observing a looking event
Explaining lookingInDEvent691
  robot is observing a looking event
Explaining lookingInDEvent693
  robot is observing a looking event
Explaining lookingInDEvent695
  robot is observing a looking event
Explaining lookingInDEvent697
  robot is observing a looking ev

  robot is observing a looking event
Explaining lookingInDEvent896
  robot is observing a looking event
Explaining lookingInDEvent900
  robot is observing a looking event
Explaining lookingInDEvent901
  robot is observing a looking event
Explaining lookingInDEvent902
  robot is observing a looking event
Explaining lookingInDEvent906
  robot is observing a looking event
Explaining lookingInDEvent908
  robot is observing a looking event
Explaining lookingInDEvent91
  robot is observing a looking event
Explaining lookingInDEvent910
  robot is observing a looking event
Explaining lookingInDEvent912
  robot is observing a looking event
Explaining lookingInDEvent914
  robot is observing a looking event
Explaining lookingInDEvent916
  robot is observing a looking event
Explaining lookingInDEvent918
  robot is observing a looking event
Explaining lookingInDEvent920
  robot is observing a looking event
Explaining lookingInDEvent922
  robot is observing a looking event
Explaining lookingInDEvent

In [2]:
facts

f = to_data_frame(facts)
events = f.groupby("subject")
f.groupby("subject").count()
events.first()
events.nth(0)

,predicate,object,reason,score,count
subject,,,,,
lookingAtPEvent100,isa,LookingAtPointEvent,None,1.0,1
lookingAtPEvent1001,isa,LookingAtPointEvent,None,1.0,1
lookingAtPEvent1003,isa,LookingAtPointEvent,None,1.0,1
lookingAtPEvent1005,isa,LookingAtPointEvent,None,1.0,1
lookingAtPEvent1007,isa,LookingAtPointEvent,None,1.0,1
...,...,...,...,...,...
speaksUttEvent448,isa,speaksUtteranceEvent,None,1.0,1
speaksUttEvent449,isa,speaksUtteranceEvent,None,1.0,1
speaksUttEvent524,isa,speaksUtteranceEvent,None,1.0,1
